In [1]:
import tellurium as te
from equilibrator_api import ComponentContribution
cc = ComponentContribution()

In [2]:
# Read the existing file
with open(r"C:\Users\mkcoo\Documents\UW\GitHub\ECFERS\src\frenda_brenda\Files\KEGG_Filtered\M0_renamed.txt", 'r') as file:
    content = file.readlines()

In [12]:
with open(r"C:\Users\mkcoo\Documents\UW\GitHub\ECFERS\src\frenda_brenda\Files\KEGG_Filtered\M0_renamed.txt", 'r') as file:
    antimony = file.read()

In [17]:
# Filter lines and extract words
filtered_words = []

# Iterate over each line
for line in content:
    if "->" in line:  # Check if the line contains "->"
        # Split the line at the colon and extract the part after it
        parts = line.split(":")
        if len(parts) > 1:
            reaction_part = parts[1].strip().replace(";", "")  # Get the reaction after the colon

            # Split the reaction at "->" into left and right parts
            left, right = reaction_part.split("->")
            left_words = set(left.split())
            right_words = set(right.split())

            # Find words common to both sides
            common_words = left_words.intersection(right_words)

            # Filter out numbers and unwanted characters
            filtered = [word for word in common_words if not (word.isdigit() or word in ["+"])]
            
            # Add to the filtered list
            filtered_words.extend(filtered)

# Display the unique words\
speciesList = list(set(filtered_words))

In [14]:
r  = te.loada(antimony)

In [19]:
# do we need to get both floating and boundary species? what is the difference?
allSpecies = r.getFloatingSpeciesIds()

In [23]:
onlySpecies = list(set(allSpecies)-set(speciesList))

In [ ]:
# this list of dbs are the options a user can choose from as databases to annotate with. they will be used for the identity variable, this should be code that lets the users input their decision
dbs = ['kegg', 'bigg.metabolite', 'chebi', 'hmdb']

In [ ]:
with open(r"C:\Users\mkcoo\Documents\UW\GitHub\ECFERS\src\frenda_brenda\Files\KEGG_Filtered\M0_renamed.txt", 'a') as file:
    file.write(f'\n')  # Add a newline before appending new content

    for spc in onlySpecies:
        relid = None
        identity = None

        for i in cc.search_compound(spc).identifiers:
            if relid is None and i.registry.namespace == 'metacyc.compound':
                relid = i.accession
                
            if identity is None and i.registry.namespace == dbs[0]:  # or whichever annotation database the user wants
                identity = i.accession
                
            # Break the loop if both relid and identity have been found
            if relid is not None and identity is not None:
                break

            # Write each new line to the file
        file.write(f'{spc} is "{relid}";\n')
        file.write(f'{spc} identity "http://identifiers.org/kegg.compound/{identity}";\n')